## Notebook for use case digital posting assistant - stage1 - Proof of Concept
### Module B accounting assignment - version for PoC
#### Ojectives
- In this module we will develop the accounting assignment regarding the given booking information from user

#### Processing steps from concept
B0 - preparation

B1 - input: get booking information business case from user

B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database 

B3 - answer: create relevant accounting assigments for business case with LLM

B4 - output: give answer with accounting assigments to user

### B0 - Setup and configuration Modul B

The following setup-steps where processed:

* B0.0 Start SAP instances
* B0.1 install py-packages
* B0.2 load env-variables from config.json-file
* B0.3 setup and test connection to HANA DB
* B0.4 setup LLM-Connection to SAP AI-HUB
* B0.5 setup embedding-model
* B0.6 create SAP HANA-VectorStore interface with table
* B0.7 setup class user interface

### B0.0 Start SAP Instances

- Start BTP Cockpit
- Start SAP Build Dev Space
- Start HANA DB

In [ ]:
# B0.1 install py-packages
# RESET KERNEL AFTER INSTALLATION

%pip install --quiet --upgrade pip

%pip install --quiet hdbcli --break-system-packages
%pip install --quiet generative-ai-hub-sdk[all] --break-system-packages
%pip install --quiet folium --break-system-packages
%pip install --quiet ipywidgets --break-system-packages
%pip install --quiet pypdf
%pip install --quiet -U ipykernel
%pip install --quiet hana-ml
%pip install --quiet langchain
%pip install --quiet sqlalchemy-hana
%pip install --quiet transformers # for anthropic models
print("py-packages installed!")

In [2]:
# B0.2 load env-variables from config.json-file
# This script loads environment variables from a JSON configuration file
# and sets them in the current environment. It raises an error if the file does not exist
# or if the JSON file is malformed.

import json
import os


def load_env_variables(config_file):
    """
    Load environment variables from a JSON configuration file.

    Args:
        config_file (str): Path to the JSON configuration file.

    Returns:
        dict: A dictionary containing the environment variables.
    """
    if not os.path.exists(config_file):
        raise FileNotFoundError(f"The configuration file {config_file} does not exist.")

    try:
        with open(config_file, "r") as file:
            env_variables = json.load(file)
    except json.JSONDecodeError as e:
        raise ValueError(
            f"Error decoding JSON from the configuration file {config_file}: {e}"
        )

    for key, value in env_variables.items():
        # Convert non-string values to strings before setting them in os.environ
        if isinstance(value, dict):
            value = json.dumps(value)  # Convert dictionaries to JSON strings
        os.environ[key] = str(value)

    return env_variables


# Load environment variables from the specified JSON configuration file
config_file = "/home/user/.aicore/config.json"
try:
    env_variables = load_env_variables(config_file)
    print(f"Loaded environment variables: {env_variables}")
except (FileNotFoundError, ValueError) as e:
    print(e)

Loaded environment variables: {'AICORE_BASE_URL': 'https://api.ai.prod-eu20.westeurope.azure.ml.hana.ondemand.com', 'AICORE_AUTH_URL': 'https://sap-genai.authentication.eu20.hana.ondemand.com', 'AICORE_CLIENT_ID': 'sb-7a06c038-4de3-4eb6-9fc5-552da47413e6!b78145|xsuaa_std!b57362', 'AICORE_CLIENT_SECRET': 'a657d1f4-3c99-42d7-9b74-103b3d8b701e$q9XVOJA5DFfy6UxcfhbUB1vk8QB0g6anRGT5SQuqOTw=', 'AICORE_RESOURCE_GROUP': 'default', 'AICORE_DEPLOYMENT_MODEL': 'gpt-4o', 'AICORE_DEPLOYMENT_MODEL_EMBEDDING': 'text-embedding-3-large', 'hdb_host_address': 'bfff8255-c34a-41cb-a822-bf9b5f56fb16.hna0.prod-eu20.hanacloud.ondemand.com', 'hdb_user': 'DBADMIN', 'hdb_password': 'DB@hmx04', 'hdb_port': '443', 'hdb_table_name': 'ACCOUNTING_ASSIGN_SUPPORT_TABLE_DBADMIN'}


In [3]:
# B0.2 Test connection with env-Variables to SAP AI core

from gen_ai_hub.proxy.native.openai import embeddings
import os

# Ensure the correct model name is used
model_embedding_name = os.getenv(
    "AICORE_DEPLOYMENT_MODEL_EMBEDDING"
)  # Default to a valid model
try:
    response = embeddings.create(
        input="SAP Generative AI Hub is awesome!",
        # deployment_id=deployment_id, # Uncomment if using a  model deployment ID
        # model_id=model_id,  # Uncomment if using a  model ID
        model_name=model_embedding_name,  # Uncomment if using a  model name
        # model_version="latest",  # Uncomment if using a specific version
        # model_type="text",  # Uncomment if using a specific model type
        # nmodel_version="latest",  # Uncomment if using the latest version
        # nmodel_type="text-embedding",  # Uncomment if using a specific model type
    )
    print(response.data)
except ValueError as e:
    print(f"Error: {e}")
    print("Ensure the model name matches an existing deployment in SAP AI Hub.")

[Embedding(embedding=[-0.007258030120283365, 0.007906136102974415, -0.02061811275780201, 0.007136747241020203, 0.017904404550790787, 0.02568167820572853, 0.008937042206525803, 0.045177921652793884, 0.035626884549856186, 0.027910253033041954, 0.03326186537742615, 0.0004453360743355006, 0.012507311068475246, 0.0026360105257481337, -0.033928923308849335, 0.006348407827317715, -0.022118989378213882, -0.022695083171129227, 0.005177268758416176, -0.023210536688566208, 0.003892426611855626, -0.02784961275756359, -0.03844671696424484, -0.0064962212927639484, -0.014235593378543854, -0.007189808413386345, -0.01534988172352314, 0.026212291792035103, -0.02565135806798935, 0.021633857861161232, -0.018783707171678543, 0.014182532206177711, 0.026560978963971138, -0.014697984792292118, 0.023149894550442696, 0.011446084827184677, -0.007174648344516754, -0.01341693289577961, 0.009930046275258064, -0.012651334516704082, 0.02493881992995739, -0.012878740206360817, 0.017343470826745033, 0.04802807420492172

In [4]:
# B0.3 Setup and test connection to HANA DB

import os

# from hana_ml import ConnectionContext
from hdbcli import dbapi

# Fetch environment variables
hdb_host_address = os.getenv("hdb_host_address")
hdb_user = os.getenv("hdb_user")
hdb_password = os.getenv("hdb_password")
hdb_port = os.getenv("hdb_port")

# Debugging: Print non-sensitive environment variables
print(f"hdb_host_address: {hdb_host_address}")
print(f"hdb_user: {hdb_user}")
print(f"hdb_port: {hdb_port}")

# Ensure variables are defined
if not all([hdb_host_address, hdb_user, hdb_password, hdb_port]):
    raise ValueError("One or more HANA DB connection parameters are missing.")

# Convert port to integer
hdb_port = int(hdb_port)

hana_connection = dbapi.connect(
    address=hdb_host_address,
    port=hdb_port,
    user=hdb_user,
    password=hdb_password,
    # encrypt=True
    autocommit=True,
    sslValidateCertificate=False,
)

hdb_host_address: bfff8255-c34a-41cb-a822-bf9b5f56fb16.hna0.prod-eu20.hanacloud.ondemand.com
hdb_user: DBADMIN
hdb_port: 443


In [ ]:
# B0.4 Setup LLM-Connection to SAP AI-HUB (Model)

import os
import re
import re
import dotenv
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.langchain.openai import OpenAI
from gen_ai_hub.proxy.langchain.amazon import ChatBedrock
from gen_ai_hub.proxy.langchain.amazon import ChatBedrockConverse
from gen_ai_hub.proxy.langchain.init_models import init_llm

# Supported models
# supportedModels openAI:
# gpt-35-turbo-16k, o3-mini, gpt-35-turbo,
# gpt-4-32k, gpt-4, gpt-4o, gpt-4o-mini,
# text-embedding-ada-002, text-embedding-3-large, text-embedding-3-small
# supportedModels Amazon Bedrock:
# supportedModels Amazon Bedrock:
# anthropic--claude-3-haiku, anthropic--claude-3-opus, anthropic--claude-3-sonnet, anthropic--claude-3.5-sonnet,
# amazon--titan-text-express, amazon--titan-text-lite, amazon--titan-embed-text
# supportedModels GCP Vertex AI:
# text-bison, chat-bison,
# textembedding-gecko-multilingual, textembedding-gecko,
# gemini-1.0-pro, gemini-1.5-pro, gemini-1.5-flash

# Lade aicore_model_name aus der Umgebungskonfiguration
aicore_model_name = str(os.getenv("AICORE_DEPLOYMENT_MODEL"))


# Überprüfe, ob die Variable definiert ist
if not aicore_model_name:
    raise ValueError(
        f"""Parameter LLM-Model-Name {aicore_model_name} fehlt in der Umgebungskonfiguration."""
    )

# Set LLM with model from config.json
llm = None

# llm = ChatOpenAI(proxy_model_name=aicore_model_name))
# llm = ChatOpenAI(
#     model_name=aicore_model_name,
#     temperature=0
# )

llm = init_llm(
    model_name=aicore_model_name,
    max_tokens=4000,  # je nach Bedarf
    temperature=0,
)

# check if the LLM is set
if not llm:
    raise ValueError(
        f"""Parameter LLM-Model-Name {aicore_model_name} fehlt in der Umgebungskonfiguration."""
    )
else:
    print(
        f"""Parameter LLM-Model-Name: {aicore_model_name} wurde erfolgreich geladen."""
    )

Parameter LLM-Model-Name: gpt-4o wurde erfolgreich geladen.


In [6]:
# B0.4 Check Setup LLM-Connection to SAP AI-HUB (Model)

from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from gen_ai_hub.proxy.langchain.init_models import init_llm

template = """Question: {question}
    Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])
question = "What is a ai-agent?"

# llm = init_llm('anthropic--claude-3-opus', max_tokens=300)
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"question": question})
print(response)

An AI agent is a software entity that is designed to perform tasks autonomously using artificial intelligence techniques. To understand what an AI agent is, let's break it down step by step:

1. **Definition of an Agent**: In computer science, an agent is a system that perceives its environment through sensors and acts upon that environment through actuators. It can make decisions and perform actions to achieve specific goals.

2. **Artificial Intelligence**: AI involves creating systems that can perform tasks that typically require human intelligence. This includes learning, reasoning, problem-solving, perception, and language understanding.

3. **AI Agent Characteristics**:
   - **Autonomy**: AI agents operate without human intervention, making decisions based on their programming and learned experiences.
   - **Adaptability**: They can learn from interactions with their environment and improve their performance over time.
   - **Goal-Oriented**: AI agents are designed to achieve spe

In [7]:
# B0.5 setup embedding-model
# Use the embedding models from SAP AI-hub for embedding.

# Initialize embeddings

from gen_ai_hub.proxy.langchain.init_models import init_embedding_model

ai_core_embedding_model_name = str(os.getenv("AICORE_DEPLOYMENT_MODEL_EMBEDDING"))

try:
    embeddings = init_embedding_model(ai_core_embedding_model_name)
    print("Embedding model initialized successfully: ", ai_core_embedding_model_name)
except Exception as e:
    print("Embedding model not initialized.")
    print(e)

Embedding model initialized successfully:  text-embedding-3-large


In [8]:
# B0.6 - create SAP HANA-VectorStore interface with table
# check table content with sap-hana-database explorer: select * from ACCOUNTING_ASSIGN_SUPPORT_TABLE_DBADMIN

from langchain_community.vectorstores.hanavector import HanaDB

vector_table_name = str(os.getenv("hdb_table_name"))

hana_database = HanaDB(
    embedding=embeddings, connection=hana_connection, table_name=vector_table_name
)

try:
    print(
        f"""
    Successfully created SAP HANA VectorStore interface: {hana_database.connection}
    and SAP HANA table: {vector_table_name}.
    """
    )
except Exception as e:
    print(e)


    Successfully created SAP HANA VectorStore interface: <dbapi.Connection Connection object : bfff8255-c34a-41cb-a822-bf9b5f56fb16.hna0.prod-eu20.hanacloud.ondemand.com,443,DBADMIN,DB@hmx04,True>
    and SAP HANA table: ACCOUNTING_ASSIGN_SUPPORT_TABLE_DBADMIN.
    


In [9]:
# check B0.6 - query to the SAP HANA table to verify embeddings

cursor = hana_connection.cursor()
sql = f'SELECT VEC_TEXT, TO_NVARCHAR(VEC_VECTOR) FROM "{hana_database.table_name}"'

cursor.execute(sql)
vectors = cursor.fetchall()

print(vectors[5:10])

# for vector in vectors:
#     print(vector)

[('Im Besonderen werden nur die Unterschiede in der Gliederung oder Bewertung der \nverschiedenen Rechnungslegungen thematisiert. Der Aufbau des Handbuchs  lehnt sich an die Gliederung der Bilanz und GuV nach Handelsrecht \nan, Unterschiede zu US -amerikanischen Vorschriften werden an den jeweiligen Positionen \nangesprochen. Zur Ermittlung von Bemessungsgrundlagen sind teilweise Bewertungsschemata \neingefügt. Im Anhang befindet sich eine Liste der Kostenstellen, die nach der Neudefinition der \nBuchungslogik ausschließlich für Buchungen im HGB-Bereich verwendet werden dürfen.', '[0.011995994,0.024822878,-0.01712415,0.02449831,0.035234984,-0.035053227,0.0023466188,0.044296894,-0.06268036,0.016137468,0.023213025,0.017877147,-0.022576874,-0.023537593,-0.024472345,-0.008776289,0.009328053,-0.010347193,0.008360844,-0.008166104,0.009769464,0.02483586,-0.052475978,-0.010197893,-0.040194366,-0.016176416,-0.02332987,-0.01450165,-0.010983344,-0.011307911,-0.016812567,-0.004543937,-0.0063258093

In [ ]:
# B0.7 - setup class user interface
# user interface version 2 (class) - function B1.1 : get booking information business case from user
# UI-class for storing input booking information business case from user


import ipywidgets as widgets
from IPython.display import display, HTML


class InputManager:
    def __init__(self):
        self.input_text = ""
        self.history = []

        # Button-Object
        self.button = widgets.Button(
            description="Senden",
            button_style="primary",
            layout=widgets.Layout(width="150px", height="40px", margin="0 0 10px 0"),
        )

        # Input-Field with Header
        input_label = widgets.HTML(
            value="<h3 style='margin-bottom: 10px; color: #333;'>Eingabe Buchungsinformationen</h3>"
        )

        # Widget for Text-input
        self.text_input = widgets.Textarea(
            value="",
            placeholder="Geben Sie hier Ihre Buchungsinformationen ein...",
            disabled=False,
            layout=widgets.Layout(
                width="800px",
                height="200px",
                # overflow='hidden'  # Verhindert Scrolling
            ),
        )

        # Output-Field with Header
        output_label = widgets.HTML(
            value="<h3 style='margin: 20px 0 10px 0; color: #333;'>Ausgabe Kontierungsinformationen</h3>"
        )

        # Output-Field function
        self.output = widgets.HTML(
            value="""
            <div style='
                border: 1px solid #ddd;
                padding: 10px;
                background-color: white;
                height: 300px;
                width: 800px;
            '>
                <p>Kontierungsinformationen werden hier ausgegeben ...</p>
            </div>
            """,
            layout=widgets.Layout(
                width="800px",
                height="150px",
                # overflow='hidden'  # Verhindert Scrolling
            ),
        )

        # Button-Callback
        self.button.on_click(self.on_button_click)

        # vertical Layout UI (VBox)
        content = widgets.VBox(
            [input_label, self.button, self.text_input, output_label, self.output],
            layout=widgets.Layout(width="900px", align_items="flex-start"),
        )

        # Container UI (VBox)
        self.container = widgets.VBox(
            [
                widgets.HTML("<h1>Digitaler Buchungsassistent - Kontierungshilfe</h1>"),
                content,
            ],
            layout=widgets.Layout(width="900px", padding="20px"),
        )

    def on_button_click(self, b):
        """Wird aufgerufen, wenn der Button geklickt wird"""
        self.input_text = self.text_input.value
        self.history.append(self.input_text)

        # Output-field refresh after click
        self.update_output(f"Eingabe: {self.input_text}")

        # clear input field after click
        # self.text_input.value = ''

    def update_output(self, text):
        """Ausgabefeld aktualisieren"""
        self.output.value = f"""
        <div style='
            border: 1px solid #ddd;
            padding: 10px;
            background-color: white;
            height: 300px;
            width: 800px;
            # overflow: hidden;
        '>
            <p>{text}</p>
        </div>
        """

    def display_widget(self):
        """Widget anzeigen"""
        display(self.container)

    def get_current_input(self):
        """Aktuelle Eingabe zurückgeben"""
        return self.input_text

    def get_history(self):
        """Eingabehistorie zurückgeben"""
        return self.history

    def save_history(self, filename="input_history.json"):
        """Historie in Datei speichern"""
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(self.history, f, ensure_ascii=False, indent=2)

    def load_history(self, filename="input_history.json"):
        """Historie aus Datei laden"""
        try:
            with open(filename, "r", encoding="utf-8") as f:
                self.history = json.load(f)
        except FileNotFoundError:
            print("Keine gespeicherte Historie gefunden.")


# Instance creation
input_manager = InputManager()

# UI call
# input_manager.display_widget()

print("Instance UI created!")

### processing functions Modul B

* B1 - input: get booking information business case from user

* B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database 

* B3 - answer: create relevant accounting assigments for business case with LLM

* B4 - output: give answer with accounting assigments to user

### function B1: get booking information business case from user
### user interface

input-example Booking information for business case from user: 

- Als Geschäftsfall soll eine Rückstellung für eine Instandhaltungsleistung im laufenden Geschäftsjahr gebucht werden.
  * Die Instandhaltungsleistung sollte für eine Maschine bis zum Jahresende des laufenden Geschäftsjahres durchgeführt werden.
  * Der Kostenvoranschlag des Anbieters für die Instandhaltungsleistung beträgt im laufenden Geschäftsjahr 20.000 Euro netto.
- Als Geschäftsfall soll weiterhin die Auflösung der Rückstellung und die Buchung der Eingangsrechnung für die Instandhaltungsmaßnahme im folgenden Geschäftsjahr gebucht werden.
  * Die Instandhaltungsmaßnahme wird im folgenden Geschäftsjahr durchgeführt und abgerechnet. Die Rechnung enthält folgende  Informationen: Kreditor Maschinen-Meyer, Rechnungsbetrag Netto 19.500 EUR, Vorsteuer-Betrag von 3.705 EUR.
- Als Geschäftsfall soll die gebildete Rückstellung in der Schlussbilanz des laufenden Geschäftsjahres gebucht werden.
- Als Geschäftsfall soll die gebildete Rückstellung in der Eröffnungsbilanz des folgenden Geschäftsjahres gebucht werden.
 


In [ ]:
# function B1: get booking information business case from user
# function B1.1: get booking information business case from user
# the function show ui and store input from user

# Input-variables: none
# Output-variables: ui_input

input_manager.display_widget()
#ui_input = input_manager.get_current_input()

### function B2: retrieve relevant accounting assignments from SAP HANA vector database
### function B3: create relevant accounting assigments for business case with LLM
#### Version 2 - RetrievalQA-Chain with Type STUFF

In [ ]:
# function B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database
# function B2.2.5 - define Prompt (version 5 - finalized version HTML)

# Input-variables: none
# Output-variables: prompt_template

from langchain.prompts import PromptTemplate

prompt_template = """

# Buchungssatz-Generator für Geschäftsfälle

## Aufgabe: Erstellung von Buchungssätzen für Geschäftsfälle anhand eines vorgegebenen Kontierungshandbuchs

Extrahiere anhand des vom Buchhalter angegebenen Geschäftsfalls die Kontierungsregeln für Konto-Soll und Konto Haben 
aus dem Kontierungshandbuch.

## Geschäftsfall:

Der Geschäftsfall wird vom Buchhalter wie folgt beschrieben:

{question}

## Kontierungsregeln:

Die Kontierungsregeln sind im Kontierungshandbuch wie folgt definiert:

{context}


## Wichtige Hinweise
Gehe für die Ermittlung des Buchungssatzes Schritt für Schritt vor und achte auf die folgenden Punkte:
- Identifiziere die Kategorie des Geschäftsfalls    
- Bestimme die wesentlichen finanziellen Merkmale und Beträge
- Wende die Kontierungsregel an
- Ermittle konkrete Konto-Nummern und -Bezeichnungen aus dem Kontext des Kontierungshandbuchs
- Bestimme die exakten Buchungsbeträge (inkl. Steuern falls zutreffend)
- Identifiziere eventuelle Besonderheiten oder Ausnahmen

## Vorgehen zur Lösung der Aufgabe

Beachte die folgenden Schritte bei der Lösung der Aufgabe

**Schritt 1 - Identifiziere die Geschäftsfall-Kategorie**

- Identifiziere Art der Transaktion und beteiligte Wirtschaftsgüter/Leistungen
- Bestimme alle relevanten Beträge inkl. Steuern
- Ermittle die Geschäftsfall-Kategorie

Beispiele für Geschäftsfall-Kategorien sind:

|Geschäftsfall-Kategorie|
|-----------------------|
|Warenverkauf auf Rechnung|
|Wareneinkauf auf Rechnung|
|Zahlung an Lieferanten|
|Zahlungseingang von Kunden|
|Lohn- und Gehaltszahlungen|
|Anschaffung von Anlagevermögen|
|Abschreibungen|
|Bildung von Rückstellungen|
|Rückstellung buchen in Schlussbilanz|
|Rückstellung buchen in Eröffnungsbilanz|
|Auflösung von Rückstellungen|
|Eingangsrechung buchen|
|Eingangsrechnung zahlen|
|Zahlung der Umsatzsteuer|

**Schritt 2 - Ermittle die Kontierungsregel anhand der Geschäftsfall-Kategorie aus dem Kontierungshandbuch.**

- Ermittle die Kontierungsregel mit Kontotyp-Soll und Kontotyp-Haben anhand der ermittelten Geschäftsfall-Kategorien
- Beachte die buchhalterischen Grundprinzipien:
    + Jede Buchung erfordert mindestens ein Soll- und Haben-Konto
    + Soll-Haben-Buchungslogik korrekt anwenden (Vermehrung Aktiva/Verminderung Passiva = SOLL; Verminderung Aktiva/Vermehrung Passiva = HABEN)
    + Buchhalterische Vollständigkeit sicherstellen (Summe SOLL = Summe HABEN)
    + Buchungssätze nach Reihenfolge der Geschäftsfälle ordnen

Beispiele von Kontierungsregeln für Geschäftsfall-Kategoreien sind:

|Geschäftsfall-Kategorie|Kontotyp-Soll|Kontotyp-Haben|
|-----------------------|----------|-----------|
|Warenverkauf auf Rechnung|Forderungen aus Lieferungen und Leistungen|Umsatzerlöse und Umsatzsteuer|
|Wareneinkauf auf Rechnung|Wareneinsatz/Materialaufwand und Vorsteuer|Verbindlichkeiten aus Lieferungen und Leistungen|
|Zahlung an Lieferanten|Verbindlichkeiten aus Lieferungen und Leistungen|Bank oder Kasse|
|Zahlungseingang von Kunden|Bank oder Kasse|Forderungen aus Lieferungen und Leistungen|
|Lohn- und Gehaltszahlungen|Personalaufwand|Bank und diverse Verbindlichkeiten (Lohnsteuer, Sozialversicherung)|
|Anschaffung von Anlagevermögen|Anlagevermögen und Vorsteuer|Bank oder Verbindlichkeiten|
|Abschreibungen|Abschreibungsaufwand|Kumulierte Abschreibungen (Wertberichtigung Anlagevermögen)|
|Bildung von Rückstellungen|Aufwand für Rückstellungen|Rückstellungen|
|Auflösung von Rückstellungen|Rückstellungen|Sonstige betriebliche Erträge oder Aufwandskonto|
|Rückstellung buchen in Schlussbilanz (Bilanz)|Rückstellungen|Schlussbilanz-Konto|
|Rückstellung buchen in Schlussbilanz (GuV)|GuV-Konto |Aufwand für Lieferung und Leistung|
|Rückstellung buchen in Eröffnungsbilanz|Eröffnungsbilanz-Konto|Aufwand für Lieferung und Leistung|
|Eingangsrechung buchen|Aufwand für Lieferung und Leistung|Verbindlichkeiten oder Kreditor|
|Eingangsrechnung zahlen |Verbindlichkeiten oder Kreditor|Bank|
|Zahlung der Umsatzsteuer|Umsatzsteuer|Bank|


**Schritt 3 - Extrahiere die relevante Kontierung für Kontotyp-Soll und Kontotyp-Haben der Kontierungsregel aus dem Kontierungshandbuch**
- Ermittle die konkreten und exakten Konto-Nummern und Konto-Bezeichnungen für Kontotyp-Soll und Kontotyp-Haben aus dem Kontierungshandbuch anhand
  der gefundenen Kontierungsregel 
- Bestimme die exakten Buchungsbeträge aus den Informationen des Geschäftsfalls
- Beachte Spezialregelungen (z.B. für Steuern, Rückstellungen, Abschreibungen)

**Schritt 4 - Prüfe die Qualität des Ergebnisses**
- Prüfe die Kontierung (Konto Soll an Konto Haben)
- Prüfe die doppelte Buchführung (Betrag Soll = Betrag Haben)
- Stelle Übereinstimmung mit gesetzlichen Anforderungen sicher
- Verifiziere die inhaltliche Korrektheit der Kontierung

**Schritt 5 - Gib die Ergebnisse aus**

- Entferne Duplikate und redundante Informationen
- Priorisiere die relevantesten und spezifischsten Kontierungen
- Strukturiere das Ergebnis klar und übersichtlich

- Gib folgende Informationen des Ergebnisses aus:
    + Geschäftsfallbezeichnung
    + Geschäftsfall-Kategorie
    + Exakte Kontierungsinformation mit Kontonummern für Konto-Soll und Konto-Haben und den Konten-Bezeichnungen
    + Soll-Haben-Beziehung mit Beträgen

- Prüfe die Ausgabe entsprechend der oben ausgeführten Schritte und achte darauf, dass: 
    die Informationen klar und strukturiert präsentiert werden,
    + die Ausgabe in HTML-Format erfolgt und keine Code-Block-Markierungen enthält,
    + die Kontierungsinformationen vollständig und korrekt sind,
    + die Ausgabe keine überflüssigen Informationen enthält,
    + die Ausgabe keine persönlichen Daten oder sensiblen Informationen enthält,
    + die Ausgabe keine nicht relevanten Informationen enthält.

- Antwortformat:
<div class="buchungssatz">
  <h2>Geschäftsfall: [PRÄZISE BEZEICHNUNG GESCHÄFTSFALL]</h2>
  <h3>Geschäftsfall-Kategorie: [GESCHÄFTSFALL-KATEGORIE]</h3>
  <div class="kontierung">
    <table>
      <tr>
        <th>Soll</th>
        <th>Haben</th>
        <th>Betrag</th>
      </tr>
      <tr>
        <td>[KONTO-NR] - [BEZEICHNUNG]</td>
        <td>[KONTO-NR] - [BEZEICHNUNG]</td>
        <td>[BETRAG] [WÄHRUNG]</td>
      </tr>
      <!-- Weitere Zeilen bei Bedarf -->
    </table>
  </div>
  <div class="erläuterung">
    <p>[KURZE BEGRÜNDUNG DER KONTIERUNG]</p>
  </div>
</div>

## Wenn keine passende Kontierung gefunden werden kann:
<div class="keine-kontierung">
  <p>Für diesen Geschäftsfall konnte keine passende Kontierung in den bereitgestellten Regeln ermittelt werden. 
  Es fehlen folgende Informationen: [FEHLENDE INFORMATIONEN]</p>
</div>
"""
prompt_template = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

print(" final Prompt HTML set")

In [ ]:
# function B2 - retrieve: retrieve relevant accounting assignments from SAP HANA vector database
# function B2.2.6 - define Prompt (version 6 - finalized version JSON)

# Input-variables: none
# Output-variables: prompt_template

from langchain.prompts import PromptTemplate

prompt_template = """

# Buchungssatz-Generator für Geschäftsfälle

## Aufgabe: Erstellung von Buchungssätzen für Geschäftsfälle anhand eines vorgegebenen Kontierungshandbuchs

Extrahiere anhand des vom Buchhalter angegebenen Geschäftsfalls die Kontierungsregeln für Konto-Soll und Konto Haben 
aus dem Kontierungshandbuch.

## Geschäftsfall:

Der Geschäftsfall wird vom Buchhalter wie folgt beschrieben:

{question}

## Kontierungsregeln:

Die Kontierungsregeln sind im Kontierungshandbuch wie folgt definiert:

{context}


## Wichtige Hinweise
Gehe für die Ermittlung des Buchungssatzes Schritt für Schritt vor und achte auf die folgenden Punkte:
- Identifiziere die Kategorie des Geschäftsfalls    
- Bestimme die wesentlichen finanziellen Merkmale und Beträge
- Wende die Kontierungsregel an
- Ermittle konkrete Konto-Nummern und -Bezeichnungen aus dem Kontext des Kontierungshandbuchs
- Bestimme die exakten Buchungsbeträge (inkl. Steuern falls zutreffend)
- Identifiziere eventuelle Besonderheiten oder Ausnahmen

## Vorgehen zur Lösung der Aufgabe

Beachte die folgenden Schritte bei der Lösung der Aufgabe

**Schritt 1 - Identifiziere die Geschäftsfall-Kategorie**

- Identifiziere Art der Transaktion und beteiligte Wirtschaftsgüter/Leistungen
- Bestimme alle relevanten Beträge inkl. Steuern
- Ermittle die Geschäftsfall-Kategorie

Beispiele für Geschäftsfall-Kategorien sind:

|Geschäftsfall-Kategorie|
|-----------------------|
|Warenverkauf auf Rechnung|
|Wareneinkauf auf Rechnung|
|Zahlung an Lieferanten|
|Zahlungseingang von Kunden|
|Lohn- und Gehaltszahlungen|
|Anschaffung von Anlagevermögen|
|Abschreibungen|
|Bildung von Rückstellungen|
|Rückstellung buchen in Schlussbilanz|
|Rückstellung buchen in Eröffnungsbilanz|
|Auflösung von Rückstellungen|
|Eingangsrechung buchen|
|Eingangsrechnung zahlen|
|Zahlung der Umsatzsteuer|

**Schritt 2 - Ermittle die Kontierungsregel anhand der Geschäftsfall-Kategorie aus dem Kontierungshandbuch.**

- Ermittle die Kontierungsregel mit Kontotyp-Soll und Kontotyp-Haben anhand der ermittelten Geschäftsfall-Kategorien
- Beachte die buchhalterischen Grundprinzipien:
    + Jede Buchung erfordert mindestens ein Soll- und Haben-Konto
    + Soll-Haben-Buchungslogik korrekt anwenden (Vermehrung Aktiva/Verminderung Passiva = SOLL; Verminderung Aktiva/Vermehrung Passiva = HABEN)
    + Buchhalterische Vollständigkeit sicherstellen (Summe SOLL = Summe HABEN)
    + Buchungssätze nach Reihenfolge der Geschäftsfälle ordnen

Beispiele von Kontierungsregeln für Geschäftsfall-Kategoreien sind:

|Geschäftsfall-Kategorie|Kontotyp-Soll|Kontotyp-Haben|
|-----------------------|----------|-----------|
|Warenverkauf auf Rechnung|Forderungen aus Lieferungen und Leistungen|Umsatzerlöse und Umsatzsteuer|
|Wareneinkauf auf Rechnung|Wareneinsatz/Materialaufwand und Vorsteuer|Verbindlichkeiten aus Lieferungen und Leistungen|
|Zahlung an Lieferanten|Verbindlichkeiten aus Lieferungen und Leistungen|Bank oder Kasse|
|Zahlungseingang von Kunden|Bank oder Kasse|Forderungen aus Lieferungen und Leistungen|
|Lohn- und Gehaltszahlungen|Personalaufwand|Bank und diverse Verbindlichkeiten (Lohnsteuer, Sozialversicherung)|
|Anschaffung von Anlagevermögen|Anlagevermögen und Vorsteuer|Bank oder Verbindlichkeiten|
|Abschreibungen|Abschreibungsaufwand|Kumulierte Abschreibungen (Wertberichtigung Anlagevermögen)|
|Bildung von Rückstellungen|Aufwand für Rückstellungen|Rückstellungen|
|Auflösung von Rückstellungen|Rückstellungen|Sonstige betriebliche Erträge oder Aufwandskonto|
|Rückstellung buchen in Schlussbilanz (Bilanz)|Rückstellungen|Schlussbilanz-Konto|
|Rückstellung buchen in Schlussbilanz (GuV)|GuV-Konto |Aufwand für Lieferung und Leistung|
|Rückstellung buchen in Eröffnungsbilanz|Eröffnungsbilanz-Konto|Aufwand für Lieferung und Leistung|
|Eingangsrechung buchen|Aufwand für Lieferung und Leistung|Verbindlichkeiten oder Kreditor|
|Eingangsrechnung zahlen |Verbindlichkeiten oder Kreditor|Bank|
|Zahlung der Umsatzsteuer|Umsatzsteuer|Bank|


**Schritt 3 - Extrahiere die relevante Kontierung für Kontotyp-Soll und Kontotyp-Haben der Kontierungsregel aus dem Kontierungshandbuch**
- Ermittle die konkreten und exakten Konto-Nummern und Konto-Bezeichnungen für Kontotyp-Soll und Kontotyp-Haben aus dem Kontierungshandbuch anhand
  der gefundenen Kontierungsregel 
- Bestimme die exakten Buchungsbeträge aus den Informationen des Geschäftsfalls
- Beachte Spezialregelungen (z.B. für Steuern, Rückstellungen, Abschreibungen)

**Schritt 4 - Prüfe die Qualität des Ergebnisses**
- Prüfe die Kontierung (Konto Soll an Konto Haben)
- Prüfe die doppelte Buchführung (Betrag Soll = Betrag Haben)
- Stelle Übereinstimmung mit gesetzlichen Anforderungen sicher
- Verifiziere die inhaltliche Korrektheit der Kontierung

**Schritt 5 - Gib die Ergebnisse aus**

- Entferne Duplikate und redundante Informationen
- Priorisiere die relevantesten und spezifischsten Kontierungen
- Strukturiere das Ergebnis klar und übersichtlich

- Gib folgende Informationen des Ergebnisses aus:
    + Geschäftsfallbezeichnung
    + Geschäftsfall-Kategorie
    + Exakte Kontierungsinformation mit Kontonummern für Konto-Soll und Konto-Haben und den Konten-Bezeichnungen
    + Soll-Haben-Beziehung mit Beträgen

- Prüfe die Ausgabe entsprechend der oben ausgeführten Schritte und achte darauf, dass: 
    die Informationen klar und strukturiert präsentiert werden,
    + die Ausgabe nur absolut valides JSON-Format ohne Markdown-Formatierung oder Codeblöcke enthält
    + nur das JSON zurückgegeben wird.
    + die Kontierungsinformationen vollständig und korrekt sind,
    + die Ausgabe keine überflüssigen Informationen enthält,
    + die Ausgabe keine persönlichen Daten oder sensiblen Informationen enthält,
    + die Ausgabe keine nicht relevanten Informationen enthält.

## Antwortformat (JSON):

Du sollst ein JSON im folgenden Format erzeugen.
Für erfolgreiche Kontierung:
{{
  "geschaeftsfall": {{
    "bezeichnung": "PRÄZISE BEZEICHNUNG",
    "buchungen": [
      {{
        "soll": {{
          "kontonummer": "KONTO-NR",
          "bezeichnung": "BEZEICHNUNG"
        }},
        "haben": {{
          "kontonummer": "KONTO-NR",
          "bezeichnung": "BEZEICHNUNG"
        }},
        "betrag": "BETRAG",
        "waehrung": "WÄHRUNG"
      }}
    ],
    "erlaeuterung": "KURZE BEGRÜNDUNG DER KONTIERUNG"
  }}
}}

## Wenn keine passende Kontierung gefunden werden kann:
{{
  "fehler": {{
    "meldung": "Keine passende Kontierung gefunden",
    "fehlende_informationen": ["INFORMATION_1", "INFORMATION_2"]
  }}
}}

"""

prompt_template = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

print(" final Prompt JSON set")

In [ ]:
# function B3.2 - answer: create relevant accounting assigments for business case with LLM

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Input-variables: prompt_template
# Output-variables: answer

# set parameters
count_retrieved_documents = 10  # hint:  smaller than 20 -> to much tokens

# set input-variable from ui
question = input_manager.get_current_input()
chain_type_kwargs = {"prompt": prompt_template}

# init retriever with SAP HANA Database
retriever = hana_database.as_retriever(search_kwargs={"k": count_retrieved_documents})


question_answer_retriever = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs=chain_type_kwargs,
    verbose=True,
)

answer = question_answer_retriever.run(question)
print(answer)

In [ ]:
# function B4 - output: give answer with accounting assigments to user
# use of user interface function B1

# set answer to output field
input_manager.update_output(answer)

# display user interface (Widget)
input_manager.display_widget()